In [1]:
import argparse
from datetime import datetime
import json
import logging
import os
import math
import sys
import time
import topologyqueries,update_topology,spanning_tree
import topo_meas_queries as queries, process_network, linknet, update_topology

from gridappsd import GridAPPSD, DifferenceBuilder, utils, topics as t
from gridappsd.topics import simulation_input_topic, simulation_log_topic, simulation_output_topic, platform_log_topic


#model_mrid = sim_request["power_system_config"]["Line_name"]
#model_mrid = "_C125761E-9C21-4CA9-9271-B168150DE276" #ieee13training
#model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node
#model_mrid = "_AAE94E4A-2465-6F5E-37B1-3E72183A4E44" #test9500
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" #ieee13assets
#model_mrid="_C1C3E687-6FFD-C753-582B-632A27E28507" # IEEE 123
#model_mrid="_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"

requestedTime=1570041125



global BaseConnDict,BaseTermDict,TermList, NodeList
global XfmrKeys,XfmrDict,SwitchKeys,SwitchDict,DG_query

os.environ['GRIDAPPSD_APPLICATION_ID'] = 'gridappsd-topology-processor'
os.environ['GRIDAPPSD_APPLICATION_STATUS'] = 'STARTED'
#os.environ['GRIDAPPSD_SIMULATION_ID'] = opts.simulation_id
os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'


gapps = GridAPPSD()
assert gapps.connected

In [2]:
class topology_processor(GridAPPSD):
    
    def __init__(self, gapps):
        self.gapps = gapps

    
    # GridAPPS-D service
    def on_message(self):
        
        Topology = topology_dict(self.gapps, model_mrid)
        #service
        
    def get_switch_areas(self, model_mrid):
        topic = '/queue/goss.gridappsd.process.request.topology.switchareas'
        DistTopo = distributed_topology(self.gapps)
        message = DistTopo.create_switch_areas(model_mrid)
        # push message to API
        return message
        
    def get_base_topology(self, model_mrid):
        Topology = topology_dict(self.gapps, model_mrid)
        
        
    def get_snapshot_topology(self, model_mrid, sim_id, timestamp):
        Topology = topology_dict(self.gapps, model_mrid)
        
        
        

In [3]:
class distributed_topology(GridAPPSD):
    
    def __init__(self, gapps):
        self.gapps = gapps
        self.Topology = topology_dict(gapps, model_mrid)
        self.MVTopology = topology_dict(gapps, model_mrid)
        self.message = {}
        topic = '/queue/goss.gridappsd.process.request.topology.switchareas'
        #self.log = gapps.get_logger()
        
    def create_switch_areas(self, model_mrid):
        
        network = process_network(gapps)
        print('Querying for power system model')
        network.build_equip_dicts(model_mrid, self.Topology)
        network.build_equip_dicts(model_mrid, self.MVTopology)

        print('Building linked lists of all equipment')
        EqTypes = ['ACLineSegment', 'PowerTransformer', 'TransformerTank', 'SynchronousMachine']
        self.Topology.build_linknet(EqTypes)
        print('Building linked lists of medium-voltage equipment')
        EqTypes = ['ACLineSegment', 'RatioTapChanger', 'SynchronousMachine']
        self.MVTopology.build_linknet(EqTypes)
        print('Processing switch-delimited areas')
        MVTree = {}
        BreakerKeys = list(self.Topology.EquipDict['Breaker'].keys())
        MVTree = self.MVTopology.spanning_tree('Breaker', BreakerKeys , MVTree, 'all')
        FuseKeys = list(self.Topology.EquipDict['Fuse'].keys())
        MVTree = self.MVTopology.spanning_tree('Fuse', FuseKeys , MVTree, 'all')
        SwitchKeys = list(self.Topology.EquipDict['LoadBreakSwitch'].keys())
        MVTree = self.MVTopology.spanning_tree('LoadBreakSwitch', SwitchKeys, MVTree,'all')
        RecloserKeys = list(self.Topology.EquipDict['Recloser'].keys())
        MVTree = self.MVTopology.spanning_tree('Recloser', RecloserKeys, MVTree, 'all')
        
        output_message = self.create_output_message(self.Topology, MVTree, model_mrid)
        message = json.dumps(output_message)
        return message
    
    def create_output_message(self,Topology, MVTree, model_mrid):
        ConnNodeDict = Topology.ConnNodeDict
        # Initialize output message structure
        DistAppStruct = {}
        DistAppStruct['feeders'] = {}
        DistAppStruct['feeders'][model_mrid] = {}
        DistAppStruct['feeders'][model_mrid]['addressable_equipment'] = []
        DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'] = []
        DistAppStruct['feeders'][model_mrid]['connectivity_node'] = []
        DistAppStruct['feeders'][model_mrid]['switch_areas'] = []
        ProcessedNodes = [] # List to keep track of which nodes have been processed
        SwitchKeys = list(MVTree.keys()) # Get list of all switching devices from all CIM classes
        # Iterate through all switches
        for i1 in range(len(SwitchKeys)):
            # Initialize switch area dictionary
            SwitchArea = {}
            SwitchArea['boundary_switches'] = []
            SwitchArea['addressable_equipment'] = []
            SwitchArea['unaddressable_equipment'] = []
            SwitchArea['connectivity_node'] = []
            SwitchArea['secondary_areas'] = []
            # Initialize secondary area dictionary
            DistArea = {}
            DistArea['distribution_transformer'] = []
            DistArea['addressable_equipment'] = []
            DistArea['unaddressable_equipment'] = []
            DistArea['connectivity_node'] = []
            DistAreaFlag1 = True
            for i2 in range(len(MVTree[SwitchKeys[i1]])):
                # Select next medium-voltage node, append to processed list
                node = MVTree[SwitchKeys[i1]][i2]
                ProcessedNodes.append(node)
                # Add all connected equipment
                SwitchArea['boundary_switches'].extend(ConnNodeDict[node]['Breaker'])
                SwitchArea['boundary_switches'].extend(ConnNodeDict[node]['Fuse'])
                SwitchArea['boundary_switches'].extend(ConnNodeDict[node]['LoadBreakSwitch'])
                SwitchArea['boundary_switches'].extend(ConnNodeDict[node]['Recloser'])
                SwitchArea['addressable_equipment'].extend(ConnNodeDict[node]['SynchronousMachine'])
                SwitchArea['addressable_equipment'].extend(ConnNodeDict[node]['PowerElectronicsConnection'])
                SwitchArea['addressable_equipment'].extend(ConnNodeDict[node]['LinearShuntCompensator'])
                SwitchArea['addressable_equipment'].extend(ConnNodeDict[node]['RatioTapChanger'])
                SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[node]['ACLineSegment'])
                SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[node]['PowerTransformer'])
                SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[node]['TransformerTank'])
                SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[node]['Measurement'])
                SwitchArea['connectivity_node'].append(ConnNodeDict[node]['name'])
                # Identify PowerTransformer and TransformerTanks for secondary areas
                DistXfmrTanks = ConnNodeDict[node]['TransformerTank'] 
                DistXfmrs = ConnNodeDict[node]['PowerTransformer']
                if DistXfmrs: # Check all PowerTransformers connected to this node
                    DistAreaFlag1 = False
                    SwitchArea['unaddressable_equipment'].extend(DistXfmrs)
                    [SwitchArea, LVNodes] = self.create_dist_area(Topology, MVTree, DistXfmrs, 'PowerTransformer', SwitchArea.copy())
                    ProcessedNodes.extend(LVNodes)
                if DistXfmrTanks: # Check all TransformerTanks connected to this node
                    DistAreaFlag1 = False
                    SwitchArea['unaddressable_equipment'].extend(DistXfmrTanks)
                    [SwitchArea, LVNodes] = self.create_dist_area(Topology, MVTree, DistXfmrTanks, 'TransformerTank', SwitchArea.copy())
                    ProcessedNodes.extend(LVNodes)

            if DistAreaFlag1: # Append empty dictionary if no secondary areas
                SwitchArea['secondary_areas'].append((DistArea.copy()))

            if SwitchArea['boundary_switches']: # Append switch area if not duplicate
                DistAppStruct['feeders'][model_mrid]['switch_areas'].append(dict(SwitchArea))
                DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(SwitchArea['boundary_switches'])
                DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(SwitchArea['unaddressable_equipment'])

        # Add missing nodes to feeder level (not in switch area or secondary area)
        AllNodes = list(ConnNodeDict.keys())
        MissingNodes = list(set(AllNodes).difference(ProcessedNodes))
        for i5 in range(len(MissingNodes)):
            node = MissingNodes[i5]
            DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(ConnNodeDict[node]['SynchronousMachine'])
            DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(ConnNodeDict[node]['PowerElectronicsConnection'])
            DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(ConnNodeDict[node]['LinearShuntCompensator'])
            DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(ConnNodeDict[node]['RatioTapChanger'])
            DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(ConnNodeDict[node]['ACLineSegment'])
            DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(ConnNodeDict[node]['PowerTransformer'])
            DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(ConnNodeDict[node]['TransformerTank'])
            DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(ConnNodeDict[node]['Measurement'])
            DistAppStruct['feeders'][model_mrid]['connectivity_node'].append(node)

        return DistAppStruct
    
    def create_dist_area(self, Topology, MVTree, Xfmrs, eqtype, SwitchArea):
        # Initialize secondary area dictionary
        ConnNodeDict = Topology.ConnNodeDict
        DistArea = {}
        DistArea['distribution_transformer'] = []
        DistArea['addressable_equipment'] = []
        DistArea['unaddressable_equipment'] = []
        DistArea['connectivity_node'] = []
        LVNodes = []
        DistAreaFlag2 = False
        # Iterate through all secondary transformers
        for i3 in range(len(Xfmrs)):
            xfmr = Xfmrs[i3]
            LVTree = Topology.spanning_tree(eqtype, [xfmr], MVTree, 'all')
            LVTreeKeys = LVTree[xfmr]
            if LVTreeKeys: 
                LVTreeKeys.pop(0) # dump first node (xfmr hi-side, duplicate)
                for i4 in range(len(LVTreeKeys)):
                    lvnode = LVTreeKeys[i4]
                    DistAreaFlag2 = True
                    LVNodes.append(lvnode)
                    DistArea['distribution_transformer'] = [xfmr]
                    DistArea['addressable_equipment'].extend(ConnNodeDict[lvnode]['SynchronousMachine'])
                    DistArea['addressable_equipment'].extend(ConnNodeDict[lvnode]['PowerElectronicsConnection'])
                    DistArea['addressable_equipment'].extend(ConnNodeDict[lvnode]['EnergyConsumer'])
                    DistArea['addressable_equipment'].extend(ConnNodeDict[lvnode]['House'])
                    DistArea['unaddressable_equipment'].extend(ConnNodeDict[lvnode]['ACLineSegment'])
                    DistArea['unaddressable_equipment'].extend(ConnNodeDict[lvnode]['Measurement'])
                    DistArea['connectivity_node'].append(lvnode)
                    SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[lvnode]['ACLineSegment'])
                    SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[lvnode]['Measurement'])
        if DistAreaFlag2: # append secondary area if not empty
            SwitchArea['secondary_areas'].append((DistArea.copy()))
        return SwitchArea, LVNodes

In [4]:
class topology_dict(GridAPPSD):
    
    def __init__(self, gapps, model_mrid):
        self.model_mrid = model_mrid
        self.gapps = gapps
        self.EquipDict = {}
        self.ConnNodeDict = {}
        self.TerminalsDict = {}
        self.NodeList = []
        self.TermList = []
        Tree = {}
    
    def build_linknet(self, EqTypes):
        # Builds LinkNet linked lists for all CIM classes specified by EqTypes
        index = 0
        counter = 0
        TerminalsDict = {}
        NodeList = []
        TermList = []

        # Build LinkNetList for all specified CIM classes:
        for i0 in range(len(EqTypes)):
            [index, counter] = self.build_class_lists(EqTypes[i0], index, counter)
        # Add floating nodes not connected to a branch:
        StartTime = time.process_time()
        AllNodes = list(self.ConnNodeDict.keys())
        MissingNodes = list(set(AllNodes).difference(self.NodeList))
        for i2 in range(len(MissingNodes)):
            node = MissingNodes[i2]
            if 'list' not in self.ConnNodeDict[node]:
                self.ConnNodeDict[node]['node'] = index+1
                self.ConnNodeDict[node]['list'] = 0
                index = index+1
                self.NodeList.append(node)
        print("Processed ", len(MissingNodes), "missing nodes in ", time.process_time() - StartTime, "seconds")


    
    def build_class_lists(self, eqtype, index, old_counter):
        i1 = -1
        StartTime = time.process_time()
        EquipKeys = list(self.EquipDict[eqtype])

        for i1 in range(len(EquipKeys)):
            # Identify nodes and terminals for readability
            term1=self.EquipDict[eqtype][EquipKeys[i1]]['term1']
            node1=self.EquipDict[eqtype][EquipKeys[i1]]['node1']
            # Create keys for new terminals
            self.TerminalsDict[term1] = {}
            self.TerminalsDict[term1]['ConnectivityNode'] = node1

            self.TermList.append(term1)
            # If node1 not in LinkNet , create new keys
            if 'node' not in self.ConnNodeDict[node1]:
                self.ConnNodeDict[node1]['node'] = index+1
                self.ConnNodeDict[node1]['list'] = 0
                index = index+1
                self.NodeList.append(node1)

            # If two-terminal device, process both terminals
            if 'node2' in self.EquipDict[eqtype][EquipKeys[i1]]:
                # Identify nodes and terminals for readability
                term2=self.EquipDict[eqtype][EquipKeys[i1]]['term2']
                node2=self.EquipDict[eqtype][EquipKeys[i1]]['node2']
                # Create keys for new terminals
                self.TerminalsDict[term2] = {}
                self.TerminalsDict[term2]['ConnectivityNode'] = node2
                self.TerminalsDict[term1]['term'] = 2*(i1+old_counter)+1
                self.TerminalsDict[term2]['term'] = 2*(i1+old_counter)+2
                self.TermList.append(term2)
                # If node2 not in LinkNet , create new keys
                if 'node' not in self.ConnNodeDict[node2]: 
                    self.ConnNodeDict[node2]['node'] = index+1
                    self.ConnNodeDict[node2]['list'] = 0
                    index = index+1
                    self.NodeList.append(node2)
                # 1. Move node list variables to terinal next    
                self.TerminalsDict[term1]['next'] = self.ConnNodeDict[node1]['list']
                self.TerminalsDict[term2]['next'] = self.ConnNodeDict[node2]['list']
                # 2. Populate Terminal list far field with nodes
                self.TerminalsDict[term1]['far'] = self.ConnNodeDict[node2]['node']
                self.TerminalsDict[term2]['far'] = self.ConnNodeDict[node1]['node']
                # 3. Populate Connectivity nodes list with terminals
                self.ConnNodeDict[node1]['list'] = self.TerminalsDict[term1]['term']
                self.ConnNodeDict[node2]['list'] = self.TerminalsDict[term2]['term']

            # If one-terminal device, process only single terminal
            else:
                self.TerminalsDict[term1]['term'] = i1+2*(old_counter)+1
                self.TerminalsDict[term1]['next'] = 0
                self.TerminalsDict[term1]['far'] = 0
                self.ConnNodeDict[node1]['list'] = self.TerminalsDict[term1]['term']

        print("Processed ", i1+1, eqtype, "objects in ", time.process_time() - StartTime, "seconds")
        counter = old_counter+i1+1
        return index, counter
    
    def spanning_tree(self, eqtype, RootKeys, Tree, Scope):

        TotalNodes=0
        old_len = len(Tree.keys())
        StartTime = time.process_time()

        # Iterate through all substation transformers
        for i6 in range(len(RootKeys)):
            key = RootKeys[i6]
            Tree[key] = []

            # If switch object, only use second node
            if eqtype in ['Breaker', 'Fuse', 'LoadBreakSwitch', 'Recloser']:
            #, 'SynchronousMachine', 'PowerElectronicsConnection']:
                    Tree[key].append(self.EquipDict[eqtype][key]['node2'])
                    FirstNode = 0
                    LastNode = 1 # only 1 node, so initialize list at 0,1
            # Otherwise, use both nodes    
            else: # Then 2-terminal object
                not_in_tree = self.check_tree(self.EquipDict[eqtype][key]['node2'], Tree, Scope, key)
                if not_in_tree:
                    Tree[key].append(self.EquipDict[eqtype][key]['node1'])
                    Tree[key].append(self.EquipDict[eqtype][key]['node2'])
                    FirstNode = 1 
                    LastNode = 2 # 2 nodes in starting list, so initialize at 1,2
                else:
                    break
            while LastNode != FirstNode:
                NextTerm = self.ConnNodeDict[Tree[key][FirstNode]]['list']
                FirstNode = FirstNode + 1
                while NextTerm != 0:
                    # Get next node and terminal for current node
                    NextNode = self.TerminalsDict[self.TermList[NextTerm-1]]['far']
                    NextTerm = self.TerminalsDict[self.TermList[NextTerm-1]]['next']
                    node = self.NodeList[NextNode-1]
                    not_in_tree = self.check_tree(node, Tree, Scope, key)
                    # Add node if not in another tree        
                    if not_in_tree:       
                        if self.ConnNodeDict[node]['nominalVoltage']:
                            # Stop building tree into sub-transmission network
                            if int(self.ConnNodeDict[node]['nominalVoltage']) < 34000: 
                                Tree[key].append(self.NodeList[NextNode-1])
                                LastNode = LastNode + 1                       
                        else: # Add node to tree if no nominal voltage defined
                            Tree[key].append(self.NodeList[NextNode-1])
                            LastNode = LastNode + 1

            NodesInTree=len(Tree[key])


            print("Processed topology from  ",  key, ' with ', NodesInTree, " buses")

            #print("Processed topology from  ", EquipDict[key]['name'], " with id ", key, ' with ', NodesInTree, " buses")
            TotalNodes=TotalNodes+NodesInTree

        print("Processed ", len(Tree.keys()) - old_len, "topology trees containing ", TotalNodes, " buses in ", time.process_time() - StartTime, "seconds")

        return Tree
    
    # function to check if a node is the spanning tree
    # use argument "all" to check all trees from all root nodes
    # use argument "single" to only check the single tree from current root node
    def check_tree(self, node, Tree, Scope, key):
        not_in_tree = True
        if Scope == 'all': 
            TreeKeys = list(Tree.keys())
            for i7 in range(len(TreeKeys)):
                if node in Tree[TreeKeys[i7]]:
                    not_in_tree = False
                    break
        else: 
            if node in Tree[key]: 
                not_in_tree = False# Check if node already in all other tree
        return not_in_tree

In [5]:
class process_network(GridAPPSD):

    def __init__(self, gapps):
        self.gapps = gapps
        
    def build_equip_dicts(self, model_mrid, Topology):

        Topology.EquipDict['ACLineSegment'] = {}
        Topology.EquipDict['Breaker'] = {}
        Topology.EquipDict['EnergyConsumer'] = {}
        Topology.EquipDict['House'] = {}
        Topology.EquipDict['Fuse'] = {}
        Topology.EquipDict['LinearShuntCompensator'] = {}
        Topology.EquipDict['LoadBreakSwitch'] = {}
        Topology.EquipDict['PowerTransformer'] = {}
        Topology.EquipDict['RatioTapChanger'] = {}
        Topology.EquipDict['Recloser'] = {}
        Topology.EquipDict['TransformerTank'] = {}    
        Topology.EquipDict['SynchronousMachine'] = {}
        Topology.EquipDict['PowerElectronicsConnection'] = {}

        # Initialize dictionary keys for all ConnectivityNode objects in model:
        StartTime = time.process_time()
        i0=-1
        NodeQuery=queries.get_all_nodes(self.gapps,model_mrid)
        for i0 in range(len(NodeQuery)):
            node=NodeQuery[i0]['cnid']['value']
            Topology.ConnNodeDict[node] = {}
            Topology.ConnNodeDict[node]['name'] = NodeQuery[i0]['busname']['value']
            Topology.ConnNodeDict[node]['TopologicalNode'] = NodeQuery[i0]['tpnid']['value']
            if 'nomv' in NodeQuery[i0]:
                Topology.ConnNodeDict[node]['nominalVoltage'] = NodeQuery[i0]['nomv']['value']
            else:
                Topology.ConnNodeDict[node]['nominalVoltage'] = []
            Topology.ConnNodeDict[node]['ACLineSegment'] = []
            Topology.ConnNodeDict[node]['Breaker'] = []
            Topology.ConnNodeDict[node]['EnergyConsumer'] = []
            Topology.ConnNodeDict[node]['Fuse'] = []
            Topology.ConnNodeDict[node]['House'] = []
            Topology.ConnNodeDict[node]['LinearShuntCompensator'] = []
            Topology.ConnNodeDict[node]['LoadBreakSwitch'] = []
            Topology.ConnNodeDict[node]['PowerTransformer'] = []
            Topology.ConnNodeDict[node]['RatioTapChanger'] = []
            Topology.ConnNodeDict[node]['Recloser'] = []
            Topology.ConnNodeDict[node]['TransformerTank'] = []    
            Topology.ConnNodeDict[node]['SynchronousMachine'] = []
            Topology.ConnNodeDict[node]['PowerElectronicsConnection'] = []
            Topology.ConnNodeDict[node]['Measurement'] = []
        print('Processed ' + str(i0+1) + ' ConnectivyNode objects in ' + str(round(1000*(time.process_time() - StartTime))) + " ms")

        # Import all measurements and associated objects:
        StartTime = time.process_time()
        MeasurementQuery=queries.get_all_measurements(self.gapps,model_mrid)
        i1 = -1
        # Parse all entries in query response
        for i1 in range(len(MeasurementQuery)):    
            node = MeasurementQuery[i1]['cnid']['value']
            eqtype = MeasurementQuery[i1]['meastype']['value']
            eqid = MeasurementQuery[i1]['eqid']['value']
            # Associate measurement mRID with ConnectivityNode
            Topology.ConnNodeDict[node]['Measurement'].append(MeasurementQuery[i1]['measid']['value'])
            # Associate equipment mRID with ConnectivityNode if not already defined by prior measurement
            if eqid not in Topology.ConnNodeDict[node][eqtype]: 
                Topology.ConnNodeDict[node][eqtype].append(eqid)
            # Create equipment dictionary entry if not already defined by prior measurement
            if eqid not in Topology.EquipDict[eqtype]: 
                Topology.EquipDict[eqtype][eqid] = {}
            # Associate ConnectivityNode with equipment mRID - FIRST PASS
            if 'node1' in Topology.EquipDict[eqtype][eqid]: # If one node already defined, then assume two-terminal branch
                if Topology.EquipDict[eqtype][eqid]['node1'] != node:
                    Topology.EquipDict[eqtype][eqid]['node2'] = node
                    Topology.EquipDict[eqtype][eqid]['term2'] = MeasurementQuery[i1]['trmid']['value']
            else: # If first node, assume that it is first node
                Topology.EquipDict[eqtype][eqid]['name'] = MeasurementQuery[i1]['eqname']['value']
                Topology.EquipDict[eqtype][eqid]['node1'] = node
                Topology.EquipDict[eqtype][eqid]['term1'] = MeasurementQuery[i1]['trmid']['value']
            # NEED TO ADD LOGIC FOR 3-WINDING TRANSFORMERS LATER
        print('Processed ' + str(i1+1) + ' Measurement objects in ' + str(round(1000*(time.process_time() - StartTime))) + " ms")

        # Import all ACLineSegment objects - SECOND PASS
        StartTime = time.process_time()
        LineQuery = queries.get_all_lines(self.gapps, model_mrid)
        i2 = -1
        eqtype = 'ACLineSegment'
        for i2 in range(len(LineQuery)):
            eqid = LineQuery[i2]['id']['value']

            # Associate equipment mRID with ConnectivityNode if not already defined by prior measurement
            if eqid not in Topology.ConnNodeDict[LineQuery[i2]['node1']['value']][eqtype]: 
                Topology.ConnNodeDict[LineQuery[i2]['node1']['value']][eqtype].append(eqid)
            if eqid not in Topology.ConnNodeDict[LineQuery[i2]['node2']['value']][eqtype]: 
                Topology.ConnNodeDict[LineQuery[i2]['node2']['value']][eqtype].append(eqid)
            Topology.EquipDict[eqtype][eqid]['term1'] = LineQuery[i2]['term1']['value']
            Topology.EquipDict[eqtype][eqid]['term2'] = LineQuery[i2]['term2']['value']
            Topology.EquipDict[eqtype][eqid]['node1'] = LineQuery[i2]['node1']['value']
            Topology.EquipDict[eqtype][eqid]['node2'] = LineQuery[i2]['node2']['value']
        print('Processed ' + str(i2+1) + ' ACLineSegment objects in ' + str(round(1000*(time.process_time() - StartTime))) + " ms")

        # Import all PowerTransformer and TransformerTank objects - SECOND PASS
        StartTime = time.process_time()
        XfmrQuery = queries.get_all_transformers(self.gapps, model_mrid)
        i2 = -1
        for i2 in range(len(XfmrQuery)):
            eqtype = XfmrQuery[i2]['class']['value']
            eqid = XfmrQuery[i2]['eqid']['value']
            seq = str(XfmrQuery[i2]['seq']['value'])
            # Check if transformer not defined when parsing measurements
            if eqid not in Topology.EquipDict[eqtype]: Topology.EquipDict[eqtype][eqid] = {}
            # Identify terminal sequence and create keys for new terminals
            Topology.EquipDict[eqtype][eqid]['bus' + seq] = XfmrQuery[i2]['bus']['value']
            Topology.EquipDict[eqtype][eqid]['term' + seq] = XfmrQuery[i2]['tid']['value']
            Topology.EquipDict[eqtype][eqid]['node' + seq] = XfmrQuery[i2]['cnid']['value']
            Topology.EquipDict[eqtype][eqid]['tname' + seq] = XfmrQuery[i2]['tname']['value']
            if eqid not in Topology.ConnNodeDict[XfmrQuery[i2]['cnid']['value']][eqtype]:
                Topology.ConnNodeDict[XfmrQuery[i2]['cnid']['value']][eqtype].append(eqid)
            if 'ratedu' in XfmrQuery[i2]: # Add rated voltage if defined
                Topology.EquipDict[eqtype][eqid]['volt' + seq] = int(float(XfmrQuery[i2]['ratedu']['value']))
            else: Topology.EquipDict[eqtype][eqid]['volt' + seq] = 0 
            if 'phs' in XfmrQuery[i2]:  # Add phase if defined
                Topology.EquipDict[eqtype][eqid]['phase' + seq] = XfmrQuery[i2]['phs']['value'] 
            else: Topology.EquipDict[eqtype][eqid]['phase' + seq] = {}
        print('Processed ' + str(i2+1) + ' Transformer objects in ' + str(round(1000*(time.process_time() - StartTime))) + " ms")

        # Import all Breaker, Fuse, LoadBreakSwitch, and Recloser objects -  SECOND PASS
        StartTime = time.process_time()
        SwitchQuery = queries.get_all_switches(self.gapps, model_mrid)
        i3 = -1
        for i3 in range(len(SwitchQuery)):
            eqid = SwitchQuery[i3]['id']['value']
            eqtype = SwitchQuery[i3]['cimtype']['value']
            # Check if switch not defined when parsing measurements
            if eqid not in Topology.EquipDict[eqtype]: Topology.EquipDict[eqtype][eqid] = {}
            Topology.EquipDict[eqtype][eqid]['term1']=SwitchQuery[i3]['term1']['value']
            Topology.EquipDict[eqtype][eqid]['term2']=SwitchQuery[i3]['term2']['value']
            Topology.EquipDict[eqtype][eqid]['node1']=SwitchQuery[i3]['node1']['value']
            Topology.EquipDict[eqtype][eqid]['node2']=SwitchQuery[i3]['node2']['value']
            # Check if switch is open or closed in base model
            if SwitchQuery[i3]['open']['value'] == 'false': 
                Topology.EquipDict[eqtype][eqid]['open'] = 1
            else: 
                Topology.EquipDict[eqtype][eqid]['open'] = 0
        print('Processed ' + str(i3+1) + ' Switch objects in ' + str(round(1000*(time.process_time() - StartTime))) + " ms")


        # Import all House objects
        StartTime = time.process_time()
        HouseQuery = queries.get_all_houses(self.gapps, model_mrid)
        i4 = -1
        for i4 in range(len(HouseQuery)):
            eqid = HouseQuery[i3]['id']['value']
            eqtype = 'House'
            # Check if house not defined when parsing measurements
            if eqid not in Topology.EquipDict[eqtype]: Topology.EquipDict[eqtype][eqid] = {}
            Topology.EquipDict[eqtype][eqid]['term1']=HouseQuery[i3]['tid']['value']
            Topology.EquipDict[eqtype][eqid]['node1']=HouseQuery[i3]['cnid']['value']

        print('Processed ' + str(i4+1) + ' House objects in ' + str(round(1000*(time.process_time() - StartTime))) + " ms")

        # Import all RatioTapChanger objects
        StartTime = time.process_time()
        TapChangerQuery = queries.get_all_tapchangers(self.gapps, model_mrid)
        eqtype = 'RatioTapChanger'
        i5 = -1
        for i5 in range(len(TapChangerQuery)):
            eqid = TapChangerQuery[i5]['pxfid']['value']
            tankid = TapChangerQuery[i5]['tankid']['value']
            pxfid = TapChangerQuery[i5]['pxfid']['value']

            if tankid in Topology.EquipDict['TransformerTank']:
                Topology.EquipDict[eqtype][eqid] = dict(Topology.EquipDict['TransformerTank'][tankid])
            elif pxfid in EquipDict['PowerTransformer']:
                Topology.EquipDict[eqtype][eqid] = dict(Topology.EquipDict['PowerTransformer'][pxfid])
            Topology.EquipDict[eqtype][eqid]['name'] = TapChangerQuery[i5]['rname']['value']
            Topology.EquipDict[eqtype][eqid]['TransformerTank'] = tankid
            Topology.EquipDict[eqtype][eqid]['PowerTransformer'] = pxfid
            Topology.EquipDict[eqtype][eqid]['node'] = TapChangerQuery[i5]['cnid']['value']
            Topology.EquipDict[eqtype][eqid]['term'] = TapChangerQuery[i5]['tid']['value']
        print('Processed ' + str(i5+1) + ' RatioTapChanger objects in ' + str(round(1000*(time.process_time() - StartTime))) + " ms")


In [6]:
TP = topology_processor(gapps)

In [7]:
message = TP.get_switch_areas(model_mrid)

Querying for power system model
Processed 22 ConnectivyNode objects in 2 ms
Processed 183 Measurement objects in 5 ms
Processed 11 ACLineSegment objects in 2 ms
Processed 14 Transformer objects in 2 ms
Processed 5 Switch objects in 2 ms
Processed 0 House objects in 2 ms
Processed 3 RatioTapChanger objects in 2 ms
Processed 22 ConnectivyNode objects in 2 ms
Processed 183 Measurement objects in 5 ms
Processed 11 ACLineSegment objects in 2 ms
Processed 14 Transformer objects in 2 ms
Processed 5 Switch objects in 2 ms
Processed 0 House objects in 1 ms
Processed 3 RatioTapChanger objects in 1 ms
Building linked lists of all equipment
Processed  11 ACLineSegment objects in  6.435600000009867e-05 seconds
Processed  2 PowerTransformer objects in  1.5074999999975525e-05 seconds
Processed  4 TransformerTank objects in  2.518500000003865e-05 seconds
Processed  0 SynchronousMachine objects in  2.5319999999995346e-06 seconds
Processed  1 missing nodes in  1.7759000000006075e-05 seconds
Building lin

In [8]:
message


'{"feeders": {"_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {"addressable_equipment": ["_52DE9189-20DC-4C73-BDEE-E960FE1F9493", "_CE5D0651-676B-4AF3-8D67-41BF1B33E30C", "_2858B6C2-0886-4269-884C-06FA8B887319", "_517413CB-6977-46FA-8911-C82332E42884", "_CE5D0651-676B-4AF3-8D67-41BF1B33E30C", "_517413CB-6977-46FA-8911-C82332E42884", "_2858B6C2-0886-4269-884C-06FA8B887319"], "unaddressable_equipment": ["_44FC5A86-A099-45B8-B847-F685C5027AFB", "_B6363F07-B1BC-420B-AA4C-A34BB8F05827", "_E2E0FC64-8D45-4C55-BDB9-EAB827A46FBC", "_7c6f94c1-1419-4582-ab2d-a0b11772c26b", "_b393e719-0981-4498-9d96-07f1be7de009", "_f11a9ad9-5b68-483b-b52f-dd4af07bb77d", "_44FC5A86-A099-45B8-B847-F685C5027AFB", "_B6363F07-B1BC-420B-AA4C-A34BB8F05827", "_E2E0FC64-8D45-4C55-BDB9-EAB827A46FBC", "_A04CDFB1-E951-4FC4-8882-0323CD70AE3C", "_44FC5A86-A099-45B8-B847-F685C5027AFB", "_B6363F07-B1BC-420B-AA4C-A34BB8F05827", "_E2E0FC64-8D45-4C55-BDB9-EAB827A46FBC", "_5ad9e784-5b6c-4f2c-bc4d-1ccc31cfd96b", "_8dc4079e-6bbb-491d-9f4c-0c

In [9]:
Topology.ConnNodeDict['_94F822E0-7130-4205-8597-B47110BBEF4B']

NameError: name 'Topology' is not defined